# Sound Demo 2
This demo illustrates how use speech recognition tools in python

```
brew install swig git python3
pip install pocketsphinx
pip3 install SpeechRecognition
```

sources: https://realpython.com/python-speech-recognition/

#### run global setup

In [ ]:
try:
    with open("../global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

#### run local setup

In [ ]:
from src.audio.mini_recorder import miniRecorder

In [ ]:
import speech_recognition as sr
import sphinxbase
import pocketsphinx

class speechToText:
    def __init__(self):
        
        self.r = sr.Recognizer()
        
    def record(self, duration=4):
        with sr.Microphone() as source:
            # adjust for ambient noise
            self.r.adjust_for_ambient_noise(source)
            print("Say something!")
            self.audio = self.r.listen(source, phrase_time_limit=duration)
            print('Thank you')
        
    def recognize(self, API):
        
        if str.lower(API) == 'google':

            # recognize speech using Google Speech Recognition
            try:
                # for testing purposes, we're just using the default API key
                # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
                # instead of `r.recognize_google(audio)`
                print("Google Speech Recognition thinks you said:\n " + self.r.recognize_google(self.audio))
            except sr.UnknownValueError:
                print("Google Speech Recognition could not understand audio")
            except sr.RequestError as e:
                print("Could not request results from Google Speech Recognition service; {0}".format(e))
        
        if str.lower(API) == 'sphinx':
            # recognize speech using Sphinx
            try:
                print("Sphinx thinks you said:\n " + self.r.recognize_sphinx(self.audio))
            except sr.UnknownValueError:
                print("Sphinx could not understand audio")
            except sr.RequestError as e:
                print("Sphinx error; {0}".format(e))
        


In [ ]:
recorder = miniRecorder(seconds=2, rate=1600)
audio = recorder.record()
recorder.write2file('/tmp/wavtest.wav')

In [ ]:
stt = speechToText()
stt.record(duration=4)
stt.recognize(API='google')
stt.recognize(API='sphinx')

## Assembly AI

Assembly AI is another API for speech recognition. Create a profile here: https://www.assemblyai.com/ (press "Get a free API token")

Log into your account. At the top of the page you will see "Your API Token: --- ". Copy and paste this token below.


In [ ]:
# AUDIO_FILE = "as_good.wav"
AUDIO_FILE = "/tmp/wavtest.wav"

In [ ]:
import assemblyai

aai = assemblyai.Client(token='fill-in')

transcript = aai.transcribe(filename=AUDIO_FILE)

In [ ]:
transcript

In [ ]:
while transcript.status != 'completed':
    transcript = transcript.get()

text = transcript.text

In [ ]:
text

In [ ]:
transcript.props()

In [ ]:
transcript.confidence

In [ ]:
from pocketsphinx import LiveSpeech
for phrase in LiveSpeech(): print(phrase)

In [ ]:
from pocketsphinx import AudioFile

# Frames per Second
fps = 100

for phrase in AudioFile(frate=fps):  # frate (default=100)
    print('-' * 28)
    print('| %5s |  %3s  |   %4s   |' % ('start', 'end', 'word'))
    print('-' * 28)
    for s in phrase.seg():
        print('| %4ss | %4ss | %8s |' % (s.start_frame / fps, s.end_frame / fps, s.word))
    print('-' * 28)

In [ ]:
from __future__ import print_function
import os
from pocketsphinx import Pocketsphinx, get_model_path, get_data_path

model_path = get_model_path()
data_path = get_data_path()

config = {
    'hmm': os.path.join(model_path, 'en-us'),
    'lm': os.path.join(model_path, 'en-us.lm.bin'),
    'dict': os.path.join(model_path, 'cmudict-en-us.dict')
}

ps = Pocketsphinx(**config)
ps.decode(
    audio_file=AUDIO_FILE,  # os.path.join(data_path, 'goforward.raw'),
    buffer_size=2048,
    no_search=False,
    full_utt=False
)


In [ ]:
print(ps.segments()) 
print('Detailed segments:', *ps.segments(detailed=True), sep='\n')

In [ ]:
print(ps.hypothesis())  
print(ps.probability()) 
print(ps.score())       
print(ps.confidence())  